In [ ]:
# Install and load libraries for image processing steps
# pip install --user landsatxplore-master.zip
from landsatxplore.api import API
from landsatxplore.earthexplorer import EarthExplorer
import os
import shutil
from datetime import date, datetime, timedelta
import zipfile
import tarfile 
import glob
import numpy as np
import pandas as pd
from simpledbf import Dbf5
import requests
import arcpy
from arcpy import env
from arcpy.sa import *
arcpy.CheckOutExtension("spatial")

In [3]:
# Search and download Landsat satellite images
def downloadlandsat(startdate, enddate):
    # Initialize a new API instance and get an access key
    username = "username"  # change your EarthExplorer username and password
    password = "password"
    api = API(username, password)
    # 22.13,113.81,22.59,114.52
    # https://github.com/yannforget/landsatxplore/blob/master/landsatxplore/api.py
    # Search for Landsat TM scenes
    scenes = api.search(
        dataset='landsat_ot_c2_l1', bbox=(113.81, 22.13, 114.52, 22.59),
        start_date=startdate,  # start_date='2014-01-01',
        end_date=enddate,   # end_date='2015-12-31',
        max_cloud_cover=20, max_results=1000
    )
    print(f"{len(scenes)} Landsat scenes found.")
    print(scenes)
    # Log out
    api.logout()
    # Downloading scenes
    if len(scenes) > 0:
        ee = EarthExplorer(username, password)
        df = pd.read_csv("D:/WaterQuality/ImageInfo.csv")
        for s in scenes:
            print(s['landsat_product_id'])
            ee.download(s['entity_id'], output_dir='D:/WaterQuality/datadownload')
            df.loc[len(df.index)] = [s['landsat_product_id'], s['start_time'].isoformat()]
        df.to_csv("D:/WaterQuality/ImageInfo.csv", index=False)
        ee.logout()

# Search and download Sentinel satellite images
def downloadsentinel(startdate, enddate):
    # WKT Representation of BBOX of AOI
    ft = "POLYGON((113.81 22.13, 114.52 22.13, 114.52 22.59, 113.81 22.59, 113.81 22.13))" 
    data_collection = "SENTINEL-2"

    def get_keycloak():
        data = {
            "client_id": "cdse-public",
            "username": "username",  # change your copernicus dataspace username and password
            "password": "password",
            "grant_type": "password",
        }
        try:
            r = requests.post(
                "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
                data=data,
            )
            r.raise_for_status()
        except Exception as e:
            raise Exception(
                f"Keycloak token creation failed. Reponse from the server was: {r.json()}"
            )
        return r.json()["access_token"]
    
    json_ = requests.get(  # cloud le 20
        f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq '{data_collection}' and OData.CSC.Intersects(area=geography'SRID=4326;{ft}') and Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/OData.CSC.DoubleAttribute/Value le 20.00) and ContentDate/Start gt {startdate}T00:00:00.000Z and ContentDate/Start lt {enddate}T23:59:00.000Z&$count=True&$top=1000"
    ).json()
    p = pd.DataFrame.from_dict(json_["value"]) # Fetch available dataset
    print(f" total Sentinel tiles found {len(p)}")
    if len(p)>0:
        # Remove L2A and UTM50 dataset
        p = p[~p["Name"].str.contains("L2A")] 
        p = p[~p["Name"].str.contains("T50Q")] 
        df = pd.read_csv("D:/WaterQuality/ImageInfo.csv")
        for i in range(len(p)):
            url_id = p["Id"].iloc[i]
            download_name = p["Name"].iloc[i].split(".")[0]
            contentdate = p["ContentDate"].iloc[i]
            print("Start download: "+str(i+1)+"/"+str(len(p))+"; "+download_name)
            keycloak_token = get_keycloak()
            url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products("+url_id+")/$value"
            headers = {"Authorization": f"Bearer {keycloak_token}"}
            session = requests.Session()
            session.headers.update(headers)
            response = session.get(url, headers=headers, stream=True)
            with open("D:/WaterQuality/datadownload/"+download_name+".zip", "wb") as file:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        file.write(chunk)
            df.loc[len(df.index)] = [download_name, contentdate["Start"]]
            print("Finish download: "+download_name)
        df.to_csv("D:/WaterQuality/ImageInfo.csv", index=False)

# Function to preprocess a single Landsat image
def preprocessLandsat(tar):
    # extract tar
    datadir = 'D:/WaterQuality/datadownload'
    os.chdir(datadir)
    file = tarfile.open(tar)
    file.extractall('extract')
    file.close()
    # run acolite
    acolitepath = "D:/WaterQuality/acolite/acolite_py_win/dist/acolite/acolite.exe"
    settingpath = "D:/WaterQuality/acolite/setting_landsat.txt"
    os.system(acolitepath+" --cli --settings="+settingpath)
    def merge_and_mask():
        # merge 7 bands
        os.chdir('atmocor')
        tiflist = glob.glob('*L2R_rhos_*.tif')
        bandorder = [2, 3, 4, 7, 8, 0, 1]
        tiflist = [tiflist[i] for i in bandorder]
        env.workspace = 'D:/WaterQuality/datadownload/atmocor'
        arcpy.CompositeBands_management(tiflist, "compbands.tif")
        # mask land and cloud
        ras = Raster("compbands.tif")
        qaband = Raster(glob.glob(datadir+'/extract/*QA_PIXEL.TIF')[0])
        qaband_m = SetNull(qaband>22200,1)
        qaband_m = FocalStatistics(qaband_m, NbrCircle(3,"CELL"), "MEAN", "NODATA") # expand radius 3
        ras_m = ExtractByMask(ras, qaband_m)
        swir = Raster("compbands.tif\Band_6")
        green = Raster("compbands.tif\Band_3")
        nir = Raster("compbands.tif\Band_5")
        red = Raster("compbands.tif\Band_4")
        ndvi1 = arcpy.sa.Float((red-nir)/(red+nir))
        ndvi1_m = SetNull(ndvi1<0,1)
        ndwi = arcpy.sa.Float((green-swir)/(green+swir))
        ndwi_m = SetNull(ndwi<0,1)
        swir_m = SetNull(swir>0.15,1)
        ras_m = ExtractByMask(ras_m, ndvi1_m)
        ras_m = ExtractByMask(ras_m, ndwi_m)
        ras_m = ExtractByMask(ras_m, swir_m)
        # reproject
        aoi = "D:/WaterQuality/aoi/aoi.shp"
        outfilename = "D:/WaterQuality/reflectance/"+tar.replace(".tar",".tif")
        arcpy.management.ProjectRaster(ras_m, "compbands_p.tif", aoi)            
        arcpy.management.Clip("compbands_p.tif", aoi, "compbands_p_c.tif",                                
                            "#", "#", "NONE","MAINTAIN_EXTENT")
        arcpy.management.Resample("compbands_p_c.tif", outfilename, 0.00028571429)
    merge_and_mask()
    # empty extract and atmocor
    def emptyfolder(folder):
        for filename in os.listdir(folder):
            file_path = os.path.join(folder, filename)    
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
    emptyfolder("D:/WaterQuality/datadownload/extract")
    emptyfolder("D:/WaterQuality/datadownload/atmocor")
    # delete tarfile
    os.chdir(datadir)
    os.unlink(tar)

# Function to preprocess a single Sentinel image
def preprocessSentinel(zipf):
    datadir = 'D:/WaterQuality/datadownload'
    os.chdir(datadir)
    with zipfile.ZipFile(zipf, 'r') as zip_ref:
        zip_ref.extractall()
    safefolder = glob.glob('*.SAFE')[0]
    # run acolite
    settingtemp = "D:/WaterQuality/acolite/setting_sentinel.txt"
    settingpath = "D:/WaterQuality/acolite/setting_sentinel2.txt"
    # Read in the file
    with open(settingtemp, 'r') as file:
        filedata = file.read()
        filedata = filedata.replace('inputfile=', 'inputfile='+os.path.join(datadir,safefolder))
    # Write the file out again
    with open(settingpath, 'w') as file:
        file.write(filedata)
    acolitepath = "D:/WaterQuality/acolite/acolite_py_win/dist/acolite/acolite.exe"
    os.system(acolitepath+" --cli --settings="+settingpath)
    def merge_and_mask():
        # merge 7 bands
        os.chdir('atmocor')
        tiflist = glob.glob('*L2R_rhos_*.tif')
        if len(tiflist)==0: # if acolite does not produce any files
            return
        bandorder = [2, 3, 4, 5, 10, 0, 1]
        tiflist = [tiflist[i] for i in bandorder]
        env.workspace = 'D:/WaterQuality/datadownload/atmocor'
        arcpy.CompositeBands_management(tiflist, "compbands.tif")
        arcpy.management.Resample("compbands.tif", "compbands_r.tif", 30)
        # mask land and cloud
        ras = Raster("compbands_r.tif")
        swir = Raster("compbands_r.tif\Band_6")
        green = Raster("compbands_r.tif\Band_3")
        nir = Raster("compbands_r.tif\Band_5")
        red = Raster("compbands_r.tif\Band_4")
        cloud_m = SetNull((red>0.2)&(nir>0.2),1)
        cloud_m = FocalStatistics(cloud_m, NbrCircle(3,"CELL"), "MEAN", "NODATA") # expand radius 3
        ndvi1 = arcpy.sa.Float((red-nir)/(red+nir))
        ndvi1_m = SetNull(ndvi1<0,1)
        ndwi2 = arcpy.sa.Float((green-swir)/(green+swir))
        ndwi2_m = SetNull(ndwi2<0,1)
        swir_m = SetNull(swir>0.15,1)
        nir_m = SetNull((nir>0.03)&(red>0.08)&(ndwi2_m==1)&(swir_m==1)&(cloud_m==1),1) # remaining haze
        nir_m = FocalStatistics(nir_m, NbrCircle(1,"CELL"), "MEAN", "NODATA") # expand radius 1
        ras_m = ExtractByMask(ras, cloud_m)
        ras_m = ExtractByMask(ras_m, ndvi1_m)
        ras_m = ExtractByMask(ras_m, ndwi2_m)
        ras_m = ExtractByMask(ras_m, swir_m)
        ras_m = ExtractByMask(ras_m, nir_m)
        # reproject
        aoi = "D:/WaterQuality/aoi/aoi.shp"
        outfilename = "D:/WaterQuality/reflectance/"+safefolder.replace(".SAFE",".tif")
        arcpy.management.ProjectRaster(ras_m, "compbands_p.tif", aoi)
        arcpy.management.Clip("compbands_p.tif", aoi, "compbands_p_c.tif",                                
                            "#", "#", "NONE","MAINTAIN_EXTENT")
        arcpy.management.Resample("compbands_p_c.tif", outfilename, 0.00028571429)
    merge_and_mask()
    # empty extract and atmocor
    def emptyfolder(folder):
        for filename in os.listdir(folder):
            file_path = os.path.join(folder, filename)    
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
    emptyfolder("D:/WaterQuality/datadownload/atmocor")
    # delete whole safefolder
    os.chdir(datadir)
    shutil.rmtree(safefolder)
    os.unlink(zipf)

# Function to preprocess all Landsat images
def preprocessLandsat_all():
    datadir = 'D:/WaterQuality/datadownload'
    os.chdir(datadir)
    tarlist = glob.glob('*.tar')
    if len(tarlist)>0:
        for tar in tarlist:
            preprocessLandsat(tar)

# Function to preprocess all Sentinel images
def preprocessSentinel_all():
    datadir = 'D:/WaterQuality/datadownload'
    os.chdir(datadir)
    ziplist = glob.glob('*.zip')
    if len(ziplist)>0:
        for zipf in ziplist:
            preprocessSentinel(zipf)

# Function to get dates in each month
def monthstart(year, month):
    from datetime import date, datetime, timedelta
    first_date = datetime(year, month, 1)
    return first_date.strftime("%Y-%m-%d")
def monthmid1(year, month):
    from datetime import date, datetime, timedelta
    mid_date = datetime(year, month, 15)
    return mid_date.strftime("%Y-%m-%d")
def monthmid2(year, month):
    from datetime import date, datetime, timedelta
    mid_date = datetime(year, month, 16)
    return mid_date.strftime("%Y-%m-%d")
def monthend(year, month):
    from datetime import date, datetime, timedelta
    if month == 12:
        last_date = datetime(year, month, 31)
    else:
        last_date = datetime(year, month + 1, 1) + timedelta(days=-1)
    return last_date.strftime("%Y-%m-%d")

# Remove Tier 2 Landsat imagery
def removeLandsatT2():
    os.chdir("D:/WaterQuality/reflectance")
    Tier2list = glob.glob('LC*T2.*')
    if len(Tier2list)>0:
        for T2file in Tier2list:
            os.unlink(T2file)

# Rename all Landsat imagery
def renameLandsat_all():
    os.chdir("D:/WaterQuality/reflectance")
    Landsatlist = glob.glob('LC*')
    for Landsatfile in Landsatlist:
        nfilename = Landsatfile[0:25]+Landsatfile[40:] # first 25 characters & from 40 to end
        os.rename(Landsatfile, nfilename)

# Rename all Sentinel imagery
def renameSentinel_all():
    os.chdir("D:/WaterQuality/reflectance")
    Sentinellist = glob.glob('S2*')
    for Sentinelfile in Sentinellist:
        nfilename = Sentinelfile[0:19]+Sentinelfile[37:44]+Sentinelfile[60:]
        if os.path.isfile(nfilename) == True:
            nfilename = Sentinelfile[0:19]+Sentinelfile[37:44]+'a'+Sentinelfile[60:]
        os.rename(Sentinelfile, nfilename)

# Mosaic tiles acquired on the same day
def mosaictiles(): 
    os.chdir("D:/WaterQuality/reflectance")
    env.workspace = "D:/WaterQuality/reflectance"
    Landsatlist = glob.glob('LC*')
    Landsatdatelist = [i[17:25] for i in Landsatlist]
    Sentinellist = glob.glob('S2*')
    Sentineldatelist = [i[11:19] for i in Sentinellist]
    datelist = sorted(list(set(Landsatdatelist+Sentineldatelist))) # get unique date
    imglist = glob.glob('*.tif')
    for d in datelist:
        img_match = [img for img in imglist if d in img]
        outfolder = "D:/WaterQuality/preprocess_finish"
        outfilename = "LandsatSentinel_"+d+".tif"
        if len(img_match)==1:
            arcpy.management.CopyRaster(img_match[0], os.path.join(outfolder, outfilename))
            arcpy.management.Delete(img_match[0]) # delete original file in reflectance folder
        if len(img_match)>1:
            arcpy.MosaicToNewRaster_management(img_match,outfolder,outfilename,"","32_BIT_FLOAT","","7","MEAN","")
            arcpy.management.Delete(img_match) # delete original file in reflectance folder
        # deleteimage_lowvalid
        img1 = os.path.join(outfolder, outfilename)
        ras_np = arcpy.RasterToNumPyArray(img1,"","","",-9999)[0]
        if (ras_np != -9999).sum() < (2390000*0.1): # largest valid count = 2390000
            arcpy.management.Delete(img1)

# Connect all functions
def download_preprocess_allimagery(startdate, enddate): # From search download to mosaic
    print("Start download Landsat")
    downloadlandsat(startdate, enddate)
    print("Start download Sentinel")
    downloadsentinel(startdate, enddate)
    print("Start preprocess Landsat")
    preprocessLandsat_all()
    print("Start preprocess Sentinel")
    preprocessSentinel_all()
    removeLandsatT2()
    renameLandsat_all()
    renameSentinel_all()
    print("Start mosaic")
    mosaictiles()

In [ ]:
# Run the function from search download to mosaic
# download_preprocess_allimagery(startdate, enddate)
download_preprocess_allimagery("2024-07-10", "2024-07-10")

In [14]:
# predict Chla
def predictChla(imgname): # name with full path
	outfolder = "D:/WaterQuality/predict"
	os.chdir(outfolder)
	env.workspace = outfolder
	# ANN layers: 14, 6, 3, 1
	p = pd.read_json("D:/WaterQuality/extract/Chla_modelweight.json", typ="series")
	# Chla ['NR_B2B4', 'NR_B2B6', 'NR_B3B6', 'NR_B3B4', 'TB_B1B2B3', 
	# 'TB_B4B5B6', 'B2_3', 'B5', 'B4_2', 'B3_3', 'TB_B2B3B4', 'B6_3', 'NR_B1B6', 'B6_2']
	outname = imgname.replace("preprocess_finish", "predict").replace("LandsatSentinel_20", "Chla_20")
	b1 = Raster(imgname+"/Band_1")*10
	b2 = Raster(imgname+"/Band_2")*10
	b3 = Raster(imgname+"/Band_3")*10
	b4 = Raster(imgname+"/Band_4")*10
	b5 = Raster(imgname+"/Band_5")*10
	b6 = Raster(imgname+"/Band_6")*10
	ras_0 = b1*0
	ras_neg1 = ras_0 - 1
	ras_1 = ras_0 + 1
	v1 = CellStatistics([CellStatistics([(b2-b4)/(b2+b4),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
	v2 = CellStatistics([CellStatistics([(b2-b6)/(b2+b6),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
	v3 = CellStatistics([CellStatistics([(b3-b6)/(b3+b6),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
	v4 = CellStatistics([CellStatistics([(b3-b4)/(b3+b4),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
	v5 = CellStatistics([CellStatistics([(((1/b1)-(1/b2))*b3),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
	v6 = CellStatistics([CellStatistics([(((1/b4)-(1/b5))*b6),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
	v7 = b2 ** 3
	v8 = b5
	v9 = b4 ** 2
	v10 = b3 ** 3
	v11 = CellStatistics([CellStatistics([(((1/b2)-(1/b3))*b4),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
	v12 = b6 ** 3
	v13 = CellStatistics([CellStatistics([(b1-b6)/(b1+b6),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
	v14 = b6 ** 2

	h1n1 = (Exp((p[1][0]+v1*p[0][0][0]+v2*p[0][1][0]+v3*p[0][2][0]+v4*p[0][3][0]+v5*p[0][4][0]+
			v6*p[0][5][0]+v7*p[0][6][0]+v8*p[0][7][0]+v9*p[0][8][0]+v10*p[0][9][0]+
			v11*p[0][10][0]+v12*p[0][11][0]+v13*p[0][12][0]+v14*p[0][13][0])*(-1))+1)**-1
	h1n2 = (Exp((p[1][1]+v1*p[0][0][1]+v2*p[0][1][1]+v3*p[0][2][1]+v4*p[0][3][1]+v5*p[0][4][1]+
			v6*p[0][5][1]+v7*p[0][6][1]+v8*p[0][7][1]+v9*p[0][8][1]+v10*p[0][9][1]+
			v11*p[0][10][1]+v12*p[0][11][1]+v13*p[0][12][1]+v14*p[0][13][1])*(-1))+1)**-1
	h1n3 = (Exp((p[1][2]+v1*p[0][0][2]+v2*p[0][1][2]+v3*p[0][2][2]+v4*p[0][3][2]+v5*p[0][4][2]+
			v6*p[0][5][2]+v7*p[0][6][2]+v8*p[0][7][2]+v9*p[0][8][2]+v10*p[0][9][2]+
			v11*p[0][10][2]+v12*p[0][11][2]+v13*p[0][12][2]+v14*p[0][13][2])*(-1))+1)**-1
	h1n4 = (Exp((p[1][3]+v1*p[0][0][3]+v2*p[0][1][3]+v3*p[0][2][3]+v4*p[0][3][3]+v5*p[0][4][3]+
			v6*p[0][5][3]+v7*p[0][6][3]+v8*p[0][7][3]+v9*p[0][8][3]+v10*p[0][9][3]+
			v11*p[0][10][3]+v12*p[0][11][3]+v13*p[0][12][3]+v14*p[0][13][3])*(-1))+1)**-1
	h1n5 = (Exp((p[1][4]+v1*p[0][0][4]+v2*p[0][1][4]+v3*p[0][2][4]+v4*p[0][3][4]+v5*p[0][4][4]+
			v6*p[0][5][4]+v7*p[0][6][4]+v8*p[0][7][4]+v9*p[0][8][4]+v10*p[0][9][4]+
			v11*p[0][10][4]+v12*p[0][11][4]+v13*p[0][12][4]+v14*p[0][13][4])*(-1))+1)**-1
	h1n6 = (Exp((p[1][5]+v1*p[0][0][5]+v2*p[0][1][5]+v3*p[0][2][5]+v4*p[0][3][5]+v5*p[0][4][5]+
			v6*p[0][5][5]+v7*p[0][6][5]+v8*p[0][7][5]+v9*p[0][8][5]+v10*p[0][9][5]+
			v11*p[0][10][5]+v12*p[0][11][5]+v13*p[0][12][5]+v14*p[0][13][5])*(-1))+1)**-1

	h2n1 = (Exp((p[3][0]+h1n1*p[2][0][0]+h1n2*p[2][1][0]+h1n3*p[2][2][0]+
			h1n4*p[2][3][0]+h1n5*p[2][4][0]+h1n6*p[2][5][0])*(-1))+1)**-1
	h2n2 = (Exp((p[3][1]+h1n1*p[2][0][1]+h1n2*p[2][1][1]+h1n3*p[2][2][1]+
			h1n4*p[2][3][1]+h1n5*p[2][4][1]+h1n6*p[2][5][1])*(-1))+1)**-1
	h2n3 = (Exp((p[3][2]+h1n1*p[2][0][2]+h1n2*p[2][1][2]+h1n3*p[2][2][2]+
			h1n4*p[2][3][2]+h1n5*p[2][4][2]+h1n6*p[2][5][2])*(-1))+1)**-1

	pred = CellStatistics([p[5][0]+h2n1*p[4][0][0]+h2n2*p[4][1][0]+h2n3*p[4][2][0],ras_0], "MAXIMUM")
	arcpy.management.CopyRaster(pred, outname)

# predict SS
def predictSS(imgname):
	outfolder = "D:/WaterQuality/predict"
	os.chdir(outfolder)
	env.workspace = outfolder
	# ANN layers: 9, 6, 3, 1
	p = pd.read_json("D:/WaterQuality/extract/SS_modelweight.json", typ="series")
	# SS ['TB_B2B3B4', 'LH_B4B5B6', 'B3_3', 'B4_2', 'LH_B5B6B7', 'TB_B3B4B5', 'NR_B5B6', 'NR_B1B4', 'B2_3']
	outname = imgname.replace("preprocess_finish", "predict").replace("LandsatSentinel_20", "SuSo_20")
	b1 = Raster(imgname+"/Band_1")*10
	b2 = Raster(imgname+"/Band_2")*10
	b3 = Raster(imgname+"/Band_3")*10
	b4 = Raster(imgname+"/Band_4")*10
	b5 = Raster(imgname+"/Band_5")*10
	b6 = Raster(imgname+"/Band_6")*10
	b7 = Raster(imgname+"/Band_7")*10
	ras_0 = b1*0
	ras_neg1 = ras_0 - 1
	ras_1 = ras_0 + 1
	v1 = CellStatistics([CellStatistics([(((1/b2)-(1/b3))*b4),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
	v2 = b5-b4-((b6-b4)*((865-660)/(1610-660)))
	v3 = b3 ** 3
	v4 = b4 ** 2
	v5 = b6-b5-((b7-b5)*((1610-865)/(2195-865)))
	v6 = CellStatistics([CellStatistics([(((1/b3)-(1/b4))*b5),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
	v7 = CellStatistics([CellStatistics([(b5-b6)/(b5+b6),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
	v8 = CellStatistics([CellStatistics([(b1-b4)/(b1+b4),ras_neg1], "MAXIMUM"),ras_1],"MINIMUM")
	v9 = b2 ** 3

	h1n1 = (Exp((p[1][0]+v1*p[0][0][0]+v2*p[0][1][0]+v3*p[0][2][0]+v4*p[0][3][0]+v5*p[0][4][0]+
			v6*p[0][5][0]+v7*p[0][6][0]+v8*p[0][7][0]+v9*p[0][8][0])*(-1))+1)**-1
	h1n2 = (Exp((p[1][1]+v1*p[0][0][1]+v2*p[0][1][1]+v3*p[0][2][1]+v4*p[0][3][1]+v5*p[0][4][1]+
			v6*p[0][5][1]+v7*p[0][6][1]+v8*p[0][7][1]+v9*p[0][8][1])*(-1))+1)**-1
	h1n3 = (Exp((p[1][2]+v1*p[0][0][2]+v2*p[0][1][2]+v3*p[0][2][2]+v4*p[0][3][2]+v5*p[0][4][2]+
			v6*p[0][5][2]+v7*p[0][6][2]+v8*p[0][7][2]+v9*p[0][8][2])*(-1))+1)**-1
	h1n4 = (Exp((p[1][3]+v1*p[0][0][3]+v2*p[0][1][3]+v3*p[0][2][3]+v4*p[0][3][3]+v5*p[0][4][3]+
			v6*p[0][5][3]+v7*p[0][6][3]+v8*p[0][7][3]+v9*p[0][8][3])*(-1))+1)**-1
	h1n5 = (Exp((p[1][4]+v1*p[0][0][4]+v2*p[0][1][4]+v3*p[0][2][4]+v4*p[0][3][4]+v5*p[0][4][4]+
			v6*p[0][5][4]+v7*p[0][6][4]+v8*p[0][7][4]+v9*p[0][8][4])*(-1))+1)**-1
	h1n6 = (Exp((p[1][5]+v1*p[0][0][5]+v2*p[0][1][5]+v3*p[0][2][5]+v4*p[0][3][5]+v5*p[0][4][5]+
			v6*p[0][5][5]+v7*p[0][6][5]+v8*p[0][7][5]+v9*p[0][8][5])*(-1))+1)**-1

	h2n1 = (Exp((p[3][0]+h1n1*p[2][0][0]+h1n2*p[2][1][0]+h1n3*p[2][2][0]+
			h1n4*p[2][3][0]+h1n5*p[2][4][0]+h1n6*p[2][5][0])*(-1))+1)**-1
	h2n2 = (Exp((p[3][1]+h1n1*p[2][0][1]+h1n2*p[2][1][1]+h1n3*p[2][2][1]+
			h1n4*p[2][3][1]+h1n5*p[2][4][1]+h1n6*p[2][5][1])*(-1))+1)**-1
	h2n3 = (Exp((p[3][2]+h1n1*p[2][0][2]+h1n2*p[2][1][2]+h1n3*p[2][2][2]+
			h1n4*p[2][3][2]+h1n5*p[2][4][2]+h1n6*p[2][5][2])*(-1))+1)**-1

	pred = CellStatistics([p[5][0]+h2n1*p[4][0][0]+h2n2*p[4][1][0]+h2n3*p[4][2][0],ras_0], "MAXIMUM")
	arcpy.management.CopyRaster(pred, outname)

def predict_ChlaSS_all():
    imglist = glob.glob('D:/WaterQuality/preprocess_finish/*.tif')
    for i in imglist:
        predictChla(i)
        predictSS(i)
        # move img to finish folder
        imgname_move = i.replace("preprocess_finish", "preprocess_finish/finish")
        arcpy.management.CopyRaster(i, imgname_move)
        arcpy.management.Delete(i)

# update predicted latestimg
def update_latestimg():
    aoi_water = "D:/WaterQuality/aoi/aoi_water.shp"
    # Chla
    oldimg = glob.glob("D:/WaterQuality/predict_display/merge_Chla_*.tif")[0]
    chlalist = glob.glob("D:/WaterQuality/predict/Chla_*.tif")
    latestimg = chlalist[len(chlalist)-1]
    latestimg = latestimg[len(latestimg)-17:len(latestimg)]
    mergeras = arcpy.management.MosaicToNewRaster([oldimg]+chlalist, "D:/WaterQuality/predict_display", "merge_"+latestimg, "", "32_BIT_FLOAT", "", 1, "LAST")
    mergeras_focal = FocalStatistics(mergeras, NbrRectangle(3,3,"CELL"), "MEDIAN")
    outname = ("D:/WaterQuality/predict_display/merge_"+latestimg).replace(".tif", "_smoothclip.tif")
    arcpy.management.Clip(mergeras_focal, "", outname, aoi_water, "", "ClippingGeometry")
    arcpy.management.Delete(oldimg)
    arcpy.management.Delete(oldimg.replace(".tif", "_smoothclip.tif"))
    # SS
    oldimg = glob.glob("D:/WaterQuality/predict_display/merge_SuSo_*.tif")[0]
    sslist = glob.glob("D:/WaterQuality/predict/SuSo_*.tif")
    latestimg = sslist[len(sslist)-1]
    latestimg = latestimg[len(latestimg)-17:len(latestimg)]
    mergeras = arcpy.management.MosaicToNewRaster([oldimg]+sslist, "D:/WaterQuality/predict_display", "merge_"+latestimg, "", "32_BIT_FLOAT", "", 1, "LAST")
    mergeras_focal = FocalStatistics(mergeras, NbrRectangle(3,3,"CELL"), "MEDIAN")
    outname = ("D:/WaterQuality/predict_display/merge_"+latestimg).replace(".tif", "_smoothclip.tif")
    arcpy.management.Clip(mergeras_focal, "", outname, aoi_water, "", "ClippingGeometry")
    arcpy.management.Delete(oldimg)
    arcpy.management.Delete(oldimg.replace(".tif", "_smoothclip.tif"))

# update datapoint to latest date
def update_datapoint():
    datapoint = "D:/WaterQuality/ArcGISPro/DataPoint.shp"
    datapoint_d1 = "D:/WaterQuality/ArcGISPro/DataPoint_d1.shp"
    datapoint_d2 = "D:/WaterQuality/ArcGISPro/DataPoint_d2.shp"
    datapoint_all = "D:/WaterQuality/ArcGISPro/DataPoint_all.shp"
    chlalist = glob.glob("D:/WaterQuality/predict/Chla_*.tif")
    sslist = glob.glob("D:/WaterQuality/predict/SuSo_*.tif")
    for i in range(0,len(chlalist)):
        chla_d1 = chlalist[i]
        ss_d1 = sslist[i]
        arcpy.management.Copy(datapoint, datapoint_d1)
        arcpy.management.Copy(datapoint, datapoint_d2)
        arcpy.sa.ExtractMultiValuesToPoints(datapoint_d1, chla_d1+" value", "BILINEAR")
        arcpy.sa.ExtractMultiValuesToPoints(datapoint_d2, ss_d1+" value", "BILINEAR")
        # remove points with no data, else add date and extract list of valid points
        newdatapt = []
        d1 = int(chla_d1[len(chla_d1)-12:len(chla_d1)-4])
        d1_month = round(d1/100)*100+15
        with arcpy.da.UpdateCursor(datapoint_d1, ["value","pt","Date","parameter","latest","DateRange"]) as cursor:
            for row in cursor:
                if row[0] < 0:
                    cursor.deleteRow()
                else:
                    newdatapt.append(row[1])
                    row[2] = d1 # Date
                    row[3] = "Chla"
                    row[4] = 1 # latest
                    row[5] = "Day"
                    cursor.updateRow(row)
        # modify values for SS
        with arcpy.da.UpdateCursor(datapoint_d2, ["value","pt","Date","parameter","latest","DateRange"]) as cursor:
            for row in cursor:
                if row[0] < 0:
                    cursor.deleteRow()
                else:
                    row[2] = d1 # Date
                    row[3] = "SS"
                    row[4] = 1 # latest
                    row[5] = "Day"
                    cursor.updateRow(row)    
        # modify latest in datapoint_all
        with arcpy.da.UpdateCursor(datapoint_all, ["pt","latest","Date","DateRange"], "latest = 1") as cursor: # where clause
            for row in cursor:
                if row[0] in newdatapt: # if latest image provides obs on this pt
                    row[1] = 0
                    if row[3]=="Month" and row[2]==d1_month: # if same month as latest image
                        row[1] = 1
                    cursor.updateRow(row)
        arcpy.management.Append(datapoint_d1, datapoint_all)
        arcpy.management.Append(datapoint_d2, datapoint_all)
        # add monthly average data
        with arcpy.da.UpdateCursor(datapoint_d1, ["Date","DateRange"]) as cursor:
            for row in cursor:
                row[0] = d1_month
                row[1] = "Month"
                cursor.updateRow(row)
        with arcpy.da.UpdateCursor(datapoint_d2, ["Date","DateRange"]) as cursor:
            for row in cursor:
                row[0] = d1_month
                row[1] = "Month"
                cursor.updateRow(row)
        arcpy.management.Append(datapoint_d1, datapoint_all)
        arcpy.management.Append(datapoint_d2, datapoint_all)
        arcpy.management.Delete(datapoint_d1)
        arcpy.management.Delete(datapoint_d2)
        # move chla and SS tif to finish
        arcpy.management.CopyRaster(chla_d1, chla_d1.replace("predict", "predict/finish"))
        arcpy.management.Delete(chla_d1)
        arcpy.management.CopyRaster(ss_d1, ss_d1.replace("predict", "predict/finish"))
        arcpy.management.Delete(ss_d1)
        print("Finish: "+ str(i+1)+"/"+str(len(chlalist)))
    # save shp as zip file
    os.chdir("D:/WaterQuality/ArcGISPro")
    datapoint_all_new = glob.glob("DataPoint_all.*")
    zipname = 'DataPoint_all_shp_to'+str(d1)+'.zip'
    with zipfile.ZipFile(zipname, 'w') as zip_object:
        for f in datapoint_all_new:
            zip_object.write(f, compress_type=zipfile.ZIP_DEFLATED)
    return zipname
# zipname = update_datapoint()

# Function to create kmz file for ArcGIS Online display
def createkmz():
    chlalayer = glob.glob("D:/WaterQuality/predict_display/merge_Chla_*_smoothclip.tif")[0]
    chlasym = "D:/WaterQuality/ArcGISPro/chla_lyr.lyrx"
    chlalayer_sym = arcpy.management.ApplySymbologyFromLayer(chlalayer, chlasym)
    os.unlink("D:/WaterQuality/ArcGISPro/chla_kmz.kmz")
    arcpy.conversion.LayerToKML(chlalayer_sym, 
                                "D:/WaterQuality/ArcGISPro/chla_kmz.kmz", 0, "NO_COMPOSITE", 
                                '113.81800000012 22.1377142789301 114.50171429609 22.5711428568601 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', 
                                4096, 96, "CLAMPED_TO_GROUND")
    sslayer = glob.glob("D:/WaterQuality/predict_display/merge_SuSo_*_smoothclip.tif")[0]
    sssym = "D:/WaterQuality/ArcGISPro/ss_lyr.lyrx"
    sslayer_sym = arcpy.management.ApplySymbologyFromLayer(sslayer, sssym)
    os.unlink("D:/WaterQuality/ArcGISPro/ss_kmz.kmz")
    arcpy.conversion.LayerToKML(sslayer_sym, 
                                "D:/WaterQuality/ArcGISPro/ss_kmz.kmz", 0, "NO_COMPOSITE", 
                                '113.81800000012 22.1377142789301 114.50171429609 22.5711428568601 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', 
                                4096, 96, "CLAMPED_TO_GROUND")

In [ ]:
# Run the functions to predict and update datapoints
predict_ChlaSS_all()
update_latestimg()
zipname = update_datapoint()
print(zipname)
createkmz()

In [ ]:
# Upload results to ArcGIS Online
import os
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection
gis = GIS(url="https://wwww.arcgis.com", username="username", password="password")  # Change to your ArcGIS Online account with publisher role
# Data point FeatureLayer and shp
DataPoint_all_shp_flc = gis.content.get("item id")  # Item id of the datapoint featre layer collection on AGOL
flc = FeatureLayerCollection.fromitem(DataPoint_all_shp_flc)
flc.manager.overwrite(os.path.join("D:/WaterQuality/ArcGISPro",zipname))
DataPoint_all_shp_s = gis.content.get("item id")  # Item id of the datapoint service on AGOL
DataPoint_all_shp_s.update(data=os.path.join("D:/WaterQuality/ArcGISPro",zipname))
# kmz
chla_kmz = gis.content.get("item id")  # Item id of the chla kmz on AGOL
chla_kmz.update(data="D:/WaterQuality/ArcGISPro/chla_kmz.kmz")
ss_kmz = gis.content.get("item id")  # Item id of the SS kmz on AGOL
ss_kmz.update(data="D:/WaterQuality/ArcGISPro/ss_kmz.kmz")